# 智谱大模型

In [10]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 智谱API

### 安装

由于智谱官方的 zhipuai 需要 pydantic v2，与 langchain的某些包（如langserve）不兼容，因此在langchain中不建议使用。<br>
我从 zhipuai 专门修改了一个 zhipuai_pydantic_v1，用来兼容 pydantic v1，所有API与 zhipuai 完全一致。

我们可以使用 zhipuai_pydantic_v1 来做接下来的所有测试。
（如果你执意要用 zhipuai，那么在使用 langchain_chinese 之前记得将其卸载）。

In [ ]:
!pip install zhipuai_pydantic_v1

### 同步调用

In [20]:
from zhipuai_pydantic_v1 import ZhipuAI
client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-4", 
    temperature = 0.95,
    messages=[
        {
            "role": "system",
            "content": "你是一个强大的助手，你的名字叫「文成」，不要啰嗦",
        },
        {
            "role": "assistant",
            "content": "我的名字是「文成」，我是一名AI助手，请向我提问。"
        },
        {
            "role": "user",
            "content": "我是小明，你叫什么名字呢？"
        },
    ],
)

print(response)

model='glm-4' created=1708402853 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='我是「文成」，很高兴为您服务，小明。有什么我可以帮您的吗？', role='assistant', tool_calls=None))] request_id='8405788625178341182' id='8405788625178341182' usage=CompletionUsage(prompt_tokens=83, completion_tokens=19, total_tokens=102)


In [23]:
from zhipuai_pydantic_v1 import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-4", 
    temperature = 0.95,
    messages=[
        {
            "role": "system",
            "content": "你是一个强大的助手，你的名字叫「文成」",
        },
        {
            "role": "assistant",
            "content": "我的名字是「文成」，我是一名AI助手，请向我提问。"
        },
        {
            "role": "user",
            "content": "我是小明，你叫什么名字呢？"
        },
    ],
)

print(response)

model='glm-4' created=1708402885 choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='我是文成，一个基于人工智能技术的助手。很高兴遇见你，小明！如果你有任何问题或需要帮助，请随时告诉我。', role='assistant', tool_calls=None))] request_id='8367747962398260138' id='8367747962398260138' usage=CompletionUsage(prompt_tokens=80, completion_tokens=29, total_tokens=109)


### 异步调用

<div class="alert-info">
    <b>注意:</b><br>
    智谱API的异步调用不支持流。
</div>

In [ ]:
import time
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.asyncCompletions.create(
    model="glm-4",  # 填写需要调用的模型名称
    messages=[
        {
            "role": "system",
            "content": "你是一个强大的助手，你的名字叫「文成」",
        },
        {
            "role": "user",
            "content": "你叫什么名字呢？"
        }
    ],
)

task_id = response.id
prev_status = ''
get_cnt = 0

while prev_status != 'SUCCESS' and task_status != 'FAILED' and get_cnt <= 40:
    result_response = client.chat.asyncCompletions.retrieve_completion_result(id=task_id)
    current_status = result_response.task_status
    if(current_status == "PROCESSING"):
        print(".", end="", flush=True)
    elif(current_status == "SUCCESS"):
        print()
        print(result_response.choices[0].message)
    prev_status = current_status

    time.sleep(2)
    get_cnt += 1

### 事件流

In [ ]:
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",  # 填写需要调用的模型名称
    messages=[
        {"role": "system", "content": "你是一个乐于解答各种问题的助手，你的任务是为用户提供专业、准确、有见地的建议。"},
        {"role": "user", "content": "我对太阳系的行星非常感兴趣，特别是土星。请提供关于土星的基本信息，包括其大小、组成、环系统和任何独特的天文现象。"},
    ],
    stream=True,
)
for chunk in response:
    # print(chunk)
    print("-" * 80)
    print(chunk.choices[0].delta.content, end="", flush=True)

### 函数回调

In [14]:
from zhipuai import ZhipuAI

client = ZhipuAI()

response = client.chat.completions.create(
    model="glm-3-turbo",
    messages = [
        {
            "role": "user",
            "content": "你能帮我查询2024年1月1日从北京南站到上海的火车票吗？"
        }
    ],
    tools = [
        {
            "type": "function",
            "function": {
                "name": "query_train_info",
                "description": "根据用户提供的信息，查询对应的车次",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "departure": {
                            "type": "string",
                            "description": "出发城市或车站",
                        },
                        "destination": {
                            "type": "string",
                            "description": "目的地城市或车站",
                        },
                        "date": {
                            "type": "string",
                            "description": "要查询的车次日期",
                        },
                    },
                    "required": ["departure", "destination", "date"],
                },
            }
        }
    ],
    tool_choice="auto",
)
print(response.choices[0].message)

content=None role='assistant' tool_calls=[CompletionMessageToolCall(id='call_8367745282338832529', function=Function(arguments='{"date":"2024-01-01","departure":"北京南站","destination":"上海"}', name='query_train_info'), type='function')]


## 作为 RunnableLambda

### 基本功能

In [97]:
from zhipuai import ZhipuAI

client = ZhipuAI()

def zhipu_chat(messages: [str]):
    response = client.chat.completions.create(
        model="glm-3-turbo",
        messages=messages,
    )
    return(response)

In [98]:
prompt = [
    {"role": "user", "content": "讲一个关于程序员的笑话"},
]

In [99]:
zhipu_chat(prompt)

Completion(model='glm-3-turbo', created=1708146156, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。', role='assistant', tool_calls=None))], request_id='8367745522857040954', id='8367745522857040954', usage=CompletionUsage(prompt_tokens=11, completion_tokens=18, total_tokens=29))

### 包装为 Runnable

In [100]:
from langchain_core.runnables import RunnableLambda

my_zhipu_chat = RunnableLambda(zhipu_chat)

In [101]:
my_zhipu_chat.invoke(prompt)

Completion(model='glm-3-turbo', created=1708146224, choices=[CompletionChoice(index=0, finish_reason='stop', message=CompletionMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。', role='assistant', tool_calls=None))], request_id='8367743907949124838', id='8367743907949124838', usage=CompletionUsage(prompt_tokens=11, completion_tokens=18, total_tokens=29))

## ZhipuAIChatTiny：最小实现

### 定义

In [107]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import (
    BaseChatModel,
    generate_from_stream,
)
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult
from typing import Any, Dict, Iterator, List, Optional, cast

In [161]:
class ZhipuAIChatTiny(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None
    
    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        try:
            # 声明 ZhipuAI 的客户端
            from zhipuai import ZhipuAI
            self.client = ZhipuAI()
        except ImportError:
            raise RuntimeError(
                "Could not import zhipuai package. "
                "Please install it via 'pip install zhipuai'"
            )

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """使用 ZhiputAI 的同步调用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
        )

        content = response.choices[0].message.content

        return ChatResult(
            generations=[ChatGeneration(message=AIMessage(content=content))]
        )

### 使用 LLM

In [162]:
llm_tiny = ZhipuAIChatTiny()

In [163]:
llm_tiny.invoke("讲一个关于程序员的一句话笑话")

AIMessage(content='为什么程序员总是携带电脑？因为他们不想被人叫做"裸奔者"。')

### 使用 LLM + OutputParser

In [164]:
chain = llm_tiny | StrOutputParser()
chain.invoke("讲一个关于程序员的一句话笑话")

'为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。'

### 使用 Prompt + LLM + OutputParser

In [165]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("讲一个关于{topic}的笑话")
chain = prompt | llm_tiny | StrOutputParser()

chain.invoke({"topic": "程序员"})

'好的，下面是一个关于程序员的笑话：\n\n为什么程序员不喜欢自然界？\n\n因为那里有太多的bugs（虫子/错误）！\n\n希望这个笑话能让您开心一下！'

### 使用流式输出

<div class="alert-warning", style="padding: 5px">
    <b>注意：</b><br>
    因为我们没有实现 stream 方法，基类中的实现将自动调用 invoke 方法，然后一次性返回结果。
</div>

In [166]:
for chunk in llm_tiny.stream("讲一个关于程序员的一句话笑话"):
    print(chunk.content, end="_", flush=True)

为什么程序员总是混淆圣诞节和万圣节？因为 Oct 31 等于 Dec 25。_

## ZhipuAIChatStream：支持流

### 定义

In [167]:
from langchain_core.callbacks import CallbackManagerForLLMRun
from langchain_core.language_models.chat_models import (
    BaseChatModel,
    generate_from_stream,
)
from langchain_core.messages import AIMessage, AIMessageChunk, BaseMessage
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult

from typing import Any, Dict, Iterator, List, Optional, cast
from langchain_core.pydantic_v1 import BaseModel, Field

In [269]:
class ZhipuAIChatStream(BaseChatModel):
    """支持最新的智谱API"""

    client: Optional[ZhipuAI] = None

    def __init__(self, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)
        try:
            # 声明 ZhipuAI 的客户端
            from zhipuai import ZhipuAI
            self.client = ZhipuAI()
        except ImportError:
            raise RuntimeError(
                "Could not import zhipuai package. "
                "Please install it via 'pip install zhipuai'"
            )

    @property
    def _llm_type(self) -> str:
        """Return the type of chat model."""
        return "zhipuai"

    def _generate(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        stream: Optional[bool] = None,
        **kwargs: Any,
    ) -> ChatResult:
        """使用 ZhiputAI 的同步调用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
            stream=False
        )

        choice = response.choices[0]

        return ChatResult(
            generations=[
                ChatGeneration(
                    message=AIMessage(content=choice.message.content),
            )],
        )

    def _stream(
        self,
        messages: List[BaseMessage],
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> Iterator[ChatGenerationChunk]:
        """使用 ZhiputAI 的事件流用"""
        prompt: List = []
        for message in messages:
            if isinstance(message, AIMessage):
                role = "assistant"
            else:  # For both HumanMessage and SystemMessage, role is 'user'
                role = "user"

            prompt.append({"role": role, "content": message.content})

        # 使用流输出
        response = self.client.chat.completions.create(
            model="glm-3-turbo",
            messages=prompt,
            stream=True
        )

        for chunk in response:
            choice = chunk.choices[0]
            yield ChatGenerationChunk(
                message=AIMessageChunk(content=choice.delta.content),
            )

### 使用 LLM

In [270]:
llm_stream = ZhipuAIChatStream()

In [271]:
llm_stream.invoke("讲一个程序员的一句话笑话")

AIMessage(content='为什么程序员总是携带电脑？因为他们不想被人称为“裸奔者”。')

### 使用流式输出

In [272]:
for chunk in llm_stream.stream("讲一个关于程序员的一句话笑话"):
    print(chunk.content, end="_", flush=True)

为什么_程序_员_总是_混淆_圣诞_节_和_万_圣_节_？_因为_ Oct_ _3_1_ _等于_ Dec_ _2_5_。__

<div class="alert-success", style="padding: 5px">
    <b>成功了！</b><br>
    现在运行上面的代码，应当可以看到流式输出！
</div>

## ChatZhipuAI：在 langchain_chinese 中完整实现

完整的实现较为繁琐，可以在前面实践的基础上补充：

- 支持所有模型参数
- 支持异步方法
- 支持事件流推送
- 支持智谱的Tool回调
- 支持内置的search工具
- 支持内置的检索工具
- 支持图片生成能力
- 支持调用中的异常
- 提供便利的bind_tools方法
- 提供基于Tool调用的Agent
- ...


In [17]:
!poetry add langchain_chinese@latest

The currently activated Python version 3.9.18 is not supported by the project (>=3.10,<3.12).
Trying to find and use a compatible version. 
Using python3 (3.10.0)
Using version ^0.2.10 for langchain-chinese

Updating dependencies
Resolving dependencies... (411.2s)//files.pythonhosted.org/packages/28/50/8ed67814241e2684369f4b8b881c7d31a0816e76c8690ea8518017a35b7e/types_python_dateutil-2.8.19.20240106-py3-none-any.whl (408.3s)4.2s) (75.6s)dependencies... (11.1s)Resolving dependencies... (27.7s)Resolving dependencies... (30.0s)Resolving dependencies... Downloading https://files.pythonhosted.org/packages/4a/20/bcded6b17aa6dec73521b60a77dffbff15cf8aedea9388663296506b7b1e/wrapt-1.14.0-cp27-cp27m-macosx_10_9_x86_64.whl (30.3s)Resolving dependencies... (31.7s)Resolving dependencies... Downloading https://files.pythonhosted.org/packages/33/47/fc483df0b7ddeee987b6ff146c879d3556fcea82cc9aa4203d16e5871c62/numpy-1.26.3-cp310-cp310-macosx_10_9_x86_64.whl   7% (32.9s)Resolving dependencies... Downloa

<div>
    <b>ChatZhipuAI的完整实现已经发布</b><br>
    我将完整的实现作为 langchain_chinese 包的一部份发布了。
</div>

**项目地址** [https://pypi.org/project/langchain_chinese/](https://pypi.org/project/langchain_chinese/)<br> 
**源代码地址** [https://github.com/arcstep/langchain_chinese](https://github.com/arcstep/langchain_chinese)

你可以通过 pip 安装：

```
pip install -U langchain_chinese
```
或

```
poetry add langchain_chinese@latest
```

In [15]:
from langchain_chinese import ChatZhipuAI

In [16]:
llm = ChatZhipuAI()

RuntimeError: Could not import zhipuai package. Please install it via 'pip install zhipuai'

In [101]:
from langchain.schema.output_parser import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个强力助手。"),
    ("assistant", "我是一名AI助手，请向我提问。"),
    ("user", "{question}")
])

#### 2024年清明假怎么安排的？

In [106]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01)
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "2024年清明假怎么安排的？"}):
    print(s, end="|", flush=True)

202|4|年的|清明节|放假|安排|如下|：|4|月|4|日至|6|日|放假|调|休|，|共计|3|天|。|具体|来说|，|4|月|7|日|（|星期|日|）|需要|上班|。|清明节|是|中国的|传统|节日|，|也是|重要的|祭祀|节日|之一|，|主要用于|祭|祖|和|扫|墓|，|以|纪念|逝|去的|亲人|。|在这一|期间|，|人们|通常会|进行|扫|墓|、|献|花|、|祭|奠|等活动|，|表达|对|逝|者的|哀|思|和|怀念|。|同时|，|清明|时节|也是|春季|踏|青|的好|时机|，|人们|会|借此|机会|外出|赏|花|、|游玩|，|享受|春天的|自然|美景|。||

In [104]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01).bind(
    tools=[{
        "type": "web_search", 
        "web_search":{
            "enable":True,
            "search_query": "国务院2024年放假安排"
        }
    }])
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "2024年清明假怎么安排的？"}):
    print(s, end="|", flush=True)

根据|国务院|的|安排|，|202|4|年|清明节|放假|调|休|如下|：|4|月|4|日至|6|日|放假|，|共|3|天|。|4|月|7|日|（|星期|日|）|上班|。|这意味着|清明节|期间|，|大家|可以从|4|月|4|日开始|连续|休息|三天|，|但是|需要在|4|月|7|日|（|周日|）|补|班|。||

In [105]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01).bind(
    tools=[{
        "type": "web_search", 
        "web_search":{
            "enable":False
        }
    }])
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "2024年清明假怎么安排的？"}):
    print(s, end="|", flush=True)

截至|我的|知识|更新|日期|（|202|3|年|），|我|无法|提供|202|4|年|清明|假的|具体|安排|，|因为这些|信息|通常|会在|当年|年初|由|相关部门|公布|。|清明节|是|中国的|传统|节日|，|也是|公众|假期|之一|，|通常|会有|1|天的|法定|假期|，|但|具体的|放假|安排|可能会|结合|周末|调|休|形成|小|长假|。

为了|获取|202|4|年|清明|假的|准确|安排|，|建议|您|在|接近|那个|时期|时|关注|官方|发布的|节假日|安排|通知|。||

#### 今天星期几？

In [107]:
llm = ChatZhipuAI(model="glm-4", temperature=0.01)
chain = (prompt | llm | StrOutputParser())
for s in chain.stream({"question": "今天星期几？"}):
    print(s, end="|", flush=True)

抱歉|，|作为一个|AI|，|我没有|实|时的|日期|和时间|信息|。|我|建议|您|查看|您的|设备|上的|日|历来|确定|今天是|星期|几|。||

In [84]:
prompt.invoke({"question": "你是谁？"})

ChatPromptValue(messages=[SystemMessage(content='你是一个强力助手。'), AIMessage(content='我是一名AI助手，请向我提问。'), HumanMessage(content='你是谁？')])

In [85]:
chain.invoke({"question": "你是谁？"})

'我是由 OpenAI 开发的一个人工智能助手，旨在帮助用户回答问题、提供信息、解决问题和执行各种任务。我的设计是为了与用户进行自然对话，并在多个领域提供支持。很高兴见到你，ChatGLM！如果有任何问题或需要帮助，请随时告诉我。'

In [341]:
abc = ZhipuAIChat(model="ABC")
abc

ZhipuAIChat(client=<zhipuai._client.ZhipuAI object at 0x1192c81c0>)

In [342]:
abc.model

'glm-3-turbo'

In [1]:
await chain.ainvoke({"question": "你是谁？"})

NameError: name 'chain' is not defined